## <center>Data Wrangling with SQL and NoSQL- Stacey Sandy</center>

This week you will be using the MovieLens 1 million ratings dataset. By the time you are finished with this assignment, you will have another SQLite database and NoSQL database to use in other classes or for projects.

Broadly, this assignment will follow the FTE's progression:

* Load MovieLens tables into SQLite (good time to find that "multiple insert")
* Create a query to retrieve reviews into a cursor
* Create a dataclass that represents a movie review
* Translate rows of the cursor into a list of MovieReview objects
* Translate the list of MovieReviews into a list of dictionaries
* Load the list of dictionaries into TinyDB (using `insert_multiple()`)

There is one important point that will need to be addressed:

* MovieLens is comprised of 3 tables:
    * Users
    * Movies
    * Reviews

One complete review consists of data from all three tables joined together. We will work through that part together. 

<hr>

## Part 1 - Storing in SQLite

In this part, you are expected to read MovieLens's README file to find information to proceed. 

<div class="alert alert-block alert-info">
<b>Hint::</b> Jupyter notebook and JupyterLabs can open it.
</div>

## <b>This is where my Week 4 assignment submittle begins:</b>

In [1]:
#Import the libraries required.
import dataset
import sqlite3

In [2]:
# Fill in between the quotes for your own system
sql_db_path = "C:/Users/Stacey/Desktop/RU/MSDE621/WeeklyContent/data_wk4"

In [3]:
#Here the input name is the movies.dat file and the db file name is the MovieLens.db in the ml_db variable name.
input_data = 'movies.dat'
ml_db = 'data_wk4/MovieLens.db'

In [4]:
# Fill in the connection string between the parentheses
db = dataset.connect("sqlite:///" + ml_db)

In [5]:
# Are these files comma-separated?
separator = '::'

In [6]:
# Get column names from the README
# Replace *'s with column names
users_head = "UserID::Gender::Age::Occupation::ZipCode".split(separator)
movies_head = "MovieID::Title::Genres".split(separator)
ratings_head = "UserID::MovieID::Rating::Timestamp".split(separator)

Before executing the next line, stop and thnk what should be output. Does the actual output match your expectation?

In [7]:
#Yake a look at column header to ensure it is what you expected of the Users data.
users_head

['UserID', 'Gender', 'Age', 'Occupation', 'ZipCode']

Now it is time to create the database tables. As mentioned, there will be three of them. Interestingly, the `USERS` table and the `MOVIES` table both have unique ID fields alread - we will have to take that into account. The `RATINGS` table, on the other hand, does not have a unique ID column, so we don't have to worry about it. 

The general, simple format to create a table is:

`table_variable = db.create_table("table_name")` . # This is what you use for ratings.

But, in the case where the data already has an ID, we have to tell DataSet about it. The general form is:

`table_variable = db.create_table("table_name", primary_id="ID_column_name", primary_type=db.types.integer)`

So, in the case of the `MOVIES` table, the `MovieID` column is the primary key.

In [8]:
#Here we add the ratings_table variable and create the ratings database.
ratings_table = db.create_table("Ratings")

In [9]:
#Here we add the users_table variable with the UserID attribute identified in the users database.
users_table = db.create_table(
    "Users", primary_id="UserID", primary_type=db.types.integer
)

In [10]:
#Same thing here, we add the movies_table variable with the MoviesID attribute identified in the movies database.
movies_table = db.create_table(
    "Movies", primary_id="MoviesID", primary_type=db.types.integer
)

You can, and probably should, put those `create_table()` function calls in `try / except` blocks.

Let's set up variables for the data file names:

In [11]:
#Step up data file name and path where data should come from.
users_file = "C:/Users/Stacey/Desktop/RU/MSDE621/WeeklyContent/data_wk4/ml-1m/users.dat"
movies_file = "C:/Users/Stacey/Desktop/RU/MSDE621/WeeklyContent/data_wk4/ml-1m/movies.dat"
ratings_file = "C:/Users/Stacey/Desktop/RU/MSDE621/WeeklyContent/data_wk4/ml-1m/ratings.dat"

OK. Here it is, the moment you've all been waiting for -- we can start stuffing data in the tables we created. 

But, before we do the first, consider these questions and write the answers below:

**Having the ID column in the data caused one difference in our table creation (vs. Week_2).**

1) Do you notice any other differences, and if so, what are they?
Well for one, the unique id (or index) was not automatically generated since we identified the UserId and MoviesId unique attributes to each data and database.

2) If there is a difference, why is it different?
As mentioned above, the uniqe id attibure is the primary difference between this data and database here versus the Week 2 assignment. In week 2 the database automatically populated the unique id field and value for the data within the database.

3) If there is a difference, how does it affect the data retrieved with a SELECT statement?
The affects to a SELECT statement is that now we have table values and column values to be identified within the SQL SELECT statement. This is only because we have unique ID values for Users and Movies within two seperate tables. Therefore, it helps identify which id we are wanting to SELECT within the SQL statement and at which point is when we should insert the correct id value from each table.

<hr>

OK... I kind of lied a little bit. There is one more thing to show you about the insert. Yu might remember that this data set is called **"ml-1m"** which stands for _MovieLens - 1 million rows_. In the grand scheme of modern data storage, 1 million rows isn't a huge number, but it **is** enough to make even a fast laptop like mine choke a bit, so we are going to use a technique that many RDBMS systems call **Bulk Insert.** 

Bulk insert is optimized for inserting large amounts of similarly-structured data. SQLite is relatively fast so let's do a quick comparison, using the user's table. After that, it will be **up to you to populate the other 2 tables,** We will also use that progress bar from the FTE, just for fun.

In [12]:
%%time
with open (users_file) as ufile:
    for line in ufile:
        u_dict = dict(zip(users_head, line.split("::")))
        users_table.insert(u_dict)

Wall time: 1min 20s


Above was an example of data INSERT individually into the users_file. <br>
he next option would be to INSERT in bulk..aka insert_many. See example that will follow for users_file again.

In [13]:
# Drop the table before trying to insert again
# You might remember how to do this from Week 2

# HINT: you need the table name, and the drop command...


# Use a for loop just in case someone snuck in a new table on us
if (len(users_table) > 0):
    for table in db.tables:
        db[table].drop()

In [14]:
#Let's check if our table exists within the database.
db.tables

[]

In [15]:
%%time
users_list = []
with open(users_file) as ufile:
    for line in ufile:
        users_list.append(dict(zip(users_head, line.split("::"))))
users_table.insert_many(users_list)

Wall time: 698 ms


As we can see from the Wall time (%%time), it took considereably faster to insert_many or Bulk INSERT rather than INSERT individually into the users_file.

<hr>

Now **YOU** can decide how you want to do the other two tables, using `insert()` or `insert_many()`.

Since there are only 2 of them, I will let you do them one by one. _Don't get used to it!_

<b>For the purpose of this assignment to learn from experience, I decided to INSERT data for the movies_file individually and bulk INSERT with insert_many for ratings_file. It made perfect sense since ratings data file was the largest and would take the longest to load the data into the data file.</b><br>
NOTE: I learned you must include any code comments before %%time for you will get an error.

In [17]:
#Insert or insert_many movies_file here
#I had an odd error in this file, the line below gets around it
#with open(movies_file, errors="ignore") as mfile:
#REMOVE above code notes before tunning %%time.

%%time
movies_list = []
with open(movies_file) as mfile:
    for line in mfile:
        movies_list.append(dict(zip(movies_head, line.split("::"))))
movies_table.insert_many(movies_list)

Wall time: 344 ms


In [18]:
##nsert_many ratings_file here
#REMOVE above cod enotes before tunning %%time.

%%time
ratings_list = []
with open(ratings_file) as rfile:
    for line in rfile:
        ratings_list.append(dict(zip(ratings_head, line.split("::"))))
ratings_table.insert_many(ratings_list)

Wall time: 1min 46s


In [19]:
#Let's check if our table exists within the database.
db.tables

['Movies', 'Ratings', 'Users']

In [20]:
#Check the number of rows in Users database.
print(f'Rows in Users database: {len(users_table)}')

Rows in Users database: 6040


In [21]:
#Check the number of rows in Movies database.
print(f'Rows in Movies database: {len(movies_table)}')

Rows in Movies database: 3883


In [22]:
#Check the number of rows in Ratings database.
print(f'Rows in Ratings database: {len(ratings_table)}')

Rows in Ratings database: 1000209


<div class="alert alert-success">
  <strong>Success!</strong> At this point you should have a working relational database containing the MovieLens data!.
</div>

<hr>

### SQL Joins

Records are divided into multiple tables due to the process of **data normalization**. We have to **join tables** in our `SELECT` queries to get one full 
movie rating. 

In general, the **left join** or **left inner join** is the most common, although there are several types. The *left* part refers to the actual layout if you were putting the printed tables side by side on your desk. A left join/left inner join means you have a table with foreign keys on the left side and you are trying to match those keys to their primary keys on the right. Let's look at an example:

<center>Movie</center>

| MovieID | Title | Genre |
|---------|-------|-------|
|1 | Toy Story (1995)  | Animation|Children's|Comedy |
|2 | Jumanji (1995) | Adventure|Children's|Fantasy |
|3 | Grumpier Old Men (1995) | Comedy|Romance |
|4 | Waiting to Exhale (1995) | Comedy|Drama |
|5 | Father of the Bride Part II (1995) | Comedy |

<center>Users</center>

| UserID | Gender | Age | Occupation | ZipCode |
|--------|--------|-----|------------|---------|
| 1 | F | 1 | 10 | 48067 |
| 2 | M | 56 | 16 | 70072 |
| 3 | M | 25 | 15 | 55117 |
| 4 | M | 45 | 7 | 02460 |
| 5 | M | 25 | 20 | 55455 |


<center>Ratings</center>

| UserID | MovieID | Rating | Timestamp|
|--------|---------|--------|----------|
| 1 | 1193 | 5 | 978300760|
| 1 | 661: | 3 | 978302109|
| 1 | 914: | 3 | 978301968|
| 1 | 3408 | 4 | 978300275|
| 1 | 2355 | 5 | 978824291|

It should be obvious in this small example that Ratings are linked to both Movie and Users through their ids. So, to get a complete rating record, we need the Movie record where the MovieIDs match and the user where the UserIDs match. In SQL that loobks like this: 

SQL keywords are in caps.

```
SELECT m.title, m.genres, u.Gender, u.Age, u.Occupation, u.ZipCode, r.Rating, r.Timestamp 
FROM movies m 
INNER JOIN ratings r ON m.MovieID = r.MovieID 
INNER JOIN users u ON r.UserID = u.UserID 
ORDER BY m.Title ASC;
```

Normally, when referencing columns from multiple tables, you have to prefix the column name with the table name, but in this case I used a shortcut -- in the FROM part, I gave each table a one-letter alias. 

Also notice the last two lines. These will put all the matching movie titles together and then alphabetize the list. 

Let's try it and see what comes out.

In [23]:
# Put the query in here. NOTE: If you break up the lines, you need 
# a "continuation character" at the end of the line. 

movie_query = "SELECT m.Title, m.Genres, u.Gender, u.Age, u.Occupation, u.ZipCode, r.Rating, r.Timestamp \
FROM movies m \
INNER JOIN ratings r ON m.MovieID = r.MovieID  \
INNER JOIN users u ON r.UserID = u.UserID \
ORDER BY m.Title ASC;"

In [24]:
# Add the command to execute a query. 
# Reference: https://dataset.readthedocs.io/en/latest/api.html#dataset.Database.query
query_result = db.query(movie_query)

In [25]:
# Convert that result into a list for ease of use.
movie_list = []
for movies in query_result:
    movie_list.append(movies)

# Print out first movie to see what is stored in the list
movie_list[0]

OrderedDict([('Title', '$1,000,000 Duck (1971)'),
             ('Genres', "Children's|Comedy\n"),
             ('Gender', 'M'),
             ('Age', '45'),
             ('Occupation', '13'),
             ('ZipCode', '52761\n'),
             ('Rating', '2'),
             ('Timestamp', '976867230\n')])

# Part 2 - Storing in TinyDB

Hopefully you remember that TinyDB inserts dictionaries as documents. This means that the data in the `movie_list` variable is in the correct form to insert. 

In [30]:
from tinydb import TinyDB, Query, where

tiny_db = TinyDB("data_wk4/ml_nosql.json")

In [31]:
#Here is an example of insert_multiple (or bulk insert in TinyDB).
tiny_db.insert_multiple(movie_list)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [32]:
#Let's check is the dictionaries were created. Guess we could have checked the head but all was not a bad output.
tiny_db.all()

[{'Title': '$1,000,000 Duck (1971)',
  'Genres': "Children's|Comedy\n",
  'Gender': 'M',
  'Age': '45',
  'Occupation': '13',
  'ZipCode': '52761\n',
  'Rating': '2',
  'Timestamp': '976867230\n'},
 {'Title': '$1,000,000 Duck (1971)',
  'Genres': "Children's|Comedy\n",
  'Gender': 'F',
  'Age': '35',
  'Occupation': '0',
  'ZipCode': '17870\n',
  'Rating': '5',
  'Timestamp': '976215651\n'},
 {'Title': '$1,000,000 Duck (1971)',
  'Genres': "Children's|Comedy\n",
  'Gender': 'M',
  'Age': '18',
  'Occupation': '4',
  'ZipCode': '76013\n',
  'Rating': '4',
  'Timestamp': '975782711\n'},
 {'Title': '$1,000,000 Duck (1971)',
  'Genres': "Children's|Comedy\n",
  'Gender': 'M',
  'Age': '18',
  'Occupation': '20',
  'ZipCode': '92026\n',
  'Rating': '1',
  'Timestamp': '999376619\n'},
 {'Title': '$1,000,000 Duck (1971)',
  'Genres': "Children's|Comedy\n",
  'Gender': 'F',
  'Age': '35',
  'Occupation': '1',
  'ZipCode': '82601\n',
  'Rating': '3',
  'Timestamp': '975093319\n'},
 {'Title': '$

In [33]:
#Check the number of rows. This should matche the above number of rows in the Ratings database. 
len(tiny_db)

1000209

<div class="alert alert-success">
  <strong>Success!</strong> At this point you should have a working NoSQL database containing the MovieLens data!.
</div>

Now we can actually start using this data. 

SQL has some aggregation functions that can be interesting. For example, to find an average of a numeric column:

`select avg(column) from table where condition;`

<div class="alert alert-info">
  <strong>Note:</strong> At this point, I'm not sure that the Dataset library gains us anything, since we are just passing straight SQL through it. You can continue to use Dataset or switch to the SQLite3 library. I'll stay with Dataset, since it is already loaded. 
</div>

We can modify our join from above to get an average rating from women for the movie "Die Hard" like this:

<b>Below was a given example of how to populate the average rating by female reviewers for the movie DIe Hard.</b><br>
Note: we will be asked to duplicate this SQL statement for male reviewers of Die Hard.

In [34]:
movie_query = "select m.title, u.Gender, avg(r.Rating)\
from movies m \
inner join ratings r on m.MovieID = r.MovieID \
inner join users u on r.UserID = u.UserID \
where u.Gender = 'F' and m.title = 'Die Hard (1988)';"

In [35]:
query_result = db.query(movie_query)

In [36]:
# A quick little list comprehension to extract the results
f_avg = [row for row in query_result]

f_avg

[OrderedDict([('Title', 'Die Hard (1988)'),
              ('Gender', 'F'),
              ('avg(r.Rating)', 3.9185667752442996)])]

In [37]:
# So, to print it nicely:
print(f"Average female rating for {f_avg[0]['Title']} is {f_avg[0]['avg(r.Rating)']}")

Average female rating for Die Hard (1988) is 3.9185667752442996


That process is slightly more manual in TinyDB. Here, we can use TinyDB's `where()` command along with `matches()` to find movies with the right title, then use a logical and `&` to limit it to women. We can also take advantage of Python's built in `sum()` and `len()` commands to help us out.

It sounds more complicated than it is. Like this:


In [38]:
female_dh_set = tiny_db.search( (where('Title').matches('Die Hard (1988)')) & (where('Gender').matches('F')) )

That gives us a list of dictionaries, prove that is true to yourself, if you need to.

The rest is simple (Remember all numbers are stored as strings!):

In [39]:
dh_avg_f = sum(int(r['Rating']) for r in female_dh_set) / len(female_dh_set)

In [40]:
print(f'Average: {dh_avg_f}')

Average: 3.7107438016528924


## Questions:

1. Using the relational database you built, compare M and F average ratings for "Die Hard."
2. Do the same comparison with the NoSQL database.
3. Do the averages match?
4. What is the age range of female reviewers of "Gone With The Wind?" (Hint: in SQL, you can use a column more than once. Hint 2: There may be built in functions that help.)

<b>Now for the <u>male</u> reviewers Die Hard average rating. </b>

In [41]:
movie_query = "select m.title, u.Gender, avg(r.Rating)\
from movies m \
inner join ratings r on m.MovieID = r.MovieID \
inner join users u on r.UserID = u.UserID \
where u.Gender = 'M' and m.title = 'Die Hard (1988)';"

In [42]:
query_result = db.query(movie_query)

In [43]:
# A quick little list comprehension to extract the results
m_avg = [row for row in query_result]

m_avg

[OrderedDict([('Title', 'Die Hard (1988)'),
              ('Gender', 'M'),
              ('avg(r.Rating)', 4.1677704194260485)])]

In [44]:
# So, to print it nicely:
print(f"Average male rating for {m_avg[0]['Title']} is {m_avg[0]['avg(r.Rating)']}")

Average male rating for Die Hard (1988) is 4.1677704194260485


Now in tinyDB, we will conduct the average male rating for the movie Die Hard:

In [45]:
male_dh_set = tiny_db.search( (where('Title').matches('Die Hard (1988)')) & (where('Gender').matches('M')) )

In [46]:
dh_avg_m = sum(int(r['Rating']) for r in male_dh_set) / len(male_dh_set)
print(f'Average: {dh_avg_m}')

Average: 3.833167825223436


Now to answer the assignment question:
3. Do the averages match?

Interestingly both the average ratings for the movie Die Hard from both female and male vary in both outputs for SQL and NoSQl (TinyDB).<br> <b>The ratings are different by roughly a .208 rating value for females and a .335 rating value for males.</b>

In [47]:
print(f"Average SQL female rating for {f_avg[0]['Title']} is {f_avg[0]['avg(r.Rating)']}")

Average SQL female rating for Die Hard (1988) is 3.9185667752442996


In [48]:
print(f'Average Die Hard rating from female viewers in NoSQL: {dh_avg_f}')

Average Die Hard rating from female viewers in NoSQL: 3.7107438016528924


In [49]:
print(f"Average SQL male rating for {m_avg[0]['Title']} is {m_avg[0]['avg(r.Rating)']}")

Average SQL male rating for Die Hard (1988) is 4.1677704194260485


In [50]:
print(f'Average Die Hard rating from male viewers in NoSQL: {dh_avg_m}')

Average Die Hard rating from male viewers in NoSQL: 3.833167825223436


<b>Now moving on to get the female age range for the movie Gone With The Wind:</b>

4. What is the age range of female reviewers of "Gone With The Wind?" (Hint: in SQL, you can use a column more than once. Hint 2: There may be built in functions that help.)

In [93]:
movie_query = "select m.title, u.Gender, min(u.Age) \
from movies m \
inner join ratings r on m.MovieID = r.MovieID \
inner join users u on r.UserID = u.UserID \
where u.Gender = 'F' and m.title = 'Gone with the Wind (1939)' \
Group by u.Age;"

In [94]:
#Check your query
print(movie_query)

select m.title, u.Gender, min(u.Age) from movies m inner join ratings r on m.MovieID = r.MovieID inner join users u on r.UserID = u.UserID where u.Gender = 'F' and m.title = 'Gone with the Wind (1939)' Group by u.Age;


In [95]:
#Variable for query result 1 on group age from minimum to maximum.
query1_result = db.query(movie_query)

In [96]:
#Reveal group by age brackets for 'Gone with the Wind' ratings from females.
f_age = [row for row in query1_result]

f_age

[OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '1')]),
 OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '18')]),
 OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '25')]),
 OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '35')]),
 OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '45')]),
 OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '50')]),
 OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('min(u.Age)', '56')])]

In [97]:
#Let's just quality check the age groups.
len(f_age)

7

In [101]:
#Confirm max age range of reviewers.

movie_query = "select m.title, u.Gender, max(u.Age) \
from movies m \
inner join ratings r on m.MovieID = r.MovieID \
inner join users u on r.UserID = u.UserID \
where u.Gender = 'F' and m.title = 'Gone with the Wind (1939)';"

In [102]:
#Variable for query result 2 on group age from maximum.
query2_result = db.query(movie_query)

In [103]:
#Reveal age brackets for 'Gone with the Wind' ratings from females.
f_age = [row for row in query2_result]

f_age

[OrderedDict([('Title', 'Gone with the Wind (1939)'),
              ('Gender', 'F'),
              ('max(u.Age)', '56')])]

<b>In final, the age range of female reviewers for "Gone With The Wind is from age 1 to 56.</b>